---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 1 - Creating and Manipulating Graphs

Eight employees at a small company were asked to choose 3 movies that they would most enjoy watching for the upcoming company movie night. These choices are stored in the file `Employee_Movie_Choices.txt`.

A second file, `Employee_Relationships.txt`, has data on the relationships between different coworkers. 

The relationship score has value of `-100` (Enemies) to `+100` (Best Friends). A value of zero means the two employees haven't interacted or are indifferent.

Both files are tab delimited.

In [10]:
import networkx as nx
import pandas as pd
import numpy as np
from networkx.algorithms import bipartite
%matplotlib inline

# This is the set of employees
employees = set(['Pablo',
                 'Lee',
                 'Georgia',
                 'Vincent',
                 'Andy',
                 'Frida',
                 'Joan',
                 'Claude'])

# This is the set of movies
movies = set(['The Shawshank Redemption',
              'Forrest Gump',
              'The Matrix',
              'Anaconda',
              'The Social Network',
              'The Godfather',
              'Monty Python and the Holy Grail',
              'Snakes on a Plane',
              'Kung Fu Panda',
              'The Dark Knight',
              'Mean Girls'])

In [11]:
# you can use the following function to plot graphs
# make sure to comment it out before submitting to the autograder
def plot_graph(G, weight_name=None):
    '''
    G: a networkx G
    weight_name: name of the attribute for plotting edge weights (if G is weighted)
    '''
    %matplotlib notebook
    import matplotlib.pyplot as plt
    
    plt.figure()
    pos = nx.spring_layout(G)
    edges = G.edges()
    weights = None
    
    if weight_name:
        weights = [int(G[u][v][weight_name]) for u,v in edges]
        labels = nx.get_edge_attributes(G,weight_name)
        nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
        nx.draw_networkx(G, pos, edges=edges, width=weights);
    else:
        nx.draw_networkx(G, pos, edges=edges);

### Question 1

Using NetworkX, load in the bipartite graph from `Employee_Movie_Choices.txt` and return that graph.

*This function should return a networkx graph with 19 nodes and 24 edges*

In [30]:
def answer_one():
        
    return nx.read_adjlist('Employee_Movie_Choices.txt', delimiter='\t', nodetype=str)

### Question 2

Using the graph from the previous question, add nodes attributes named `'type'` where movies have the value `'movie'` and employees have the value `'employee'` and return that graph.

*This function should return a networkx graph with node attributes `{'type': 'movie'}` or `{'type': 'employee'}`*

In [116]:
def answer_two():
    # Your Code Here
    G = answer_one()
    for node in G.nodes():
        if node in employees:
            G.nodes[node].update({'type':'employee'})  
        elif node in movies:
            G.nodes[node].update({'type':'movie'})  
    return G

### Question 3

Find a weighted projection of the graph from `answer_two` which tells us how many movies different pairs of employees have in common.

*This function should return a weighted projected graph.*

In [126]:
def answer_three():
        
    G = answer_two()
    bi_w_proj_graph = bipartite.weighted_projected_graph(G, employees)
    return bi_w_proj_graph

### Question 4

Suppose you'd like to find out if people that have a high relationship score also like the same types of movies.

Find the Pearson correlation ( using `DataFrame.corr()` ) between employee relationship scores and the number of movies they have in common. If two employees have no movies in common it should be treated as a 0, not a missing value, and should be included in the correlation calculation.

*This function should return a float.*

In [225]:
G4 = pd.DataFrame(answer_three().edges(data=True), columns=['Emp1', 'Emp2', 'movies_in_common'])
G4['movies_in_common'] = G4['movies_in_common'].map(lambda x: x['weight'])
G4_duplicate = G4.copy()
G4_duplicate.rename(columns={'Emp1': 'Emp2', 'Emp2': 'Emp1'}, inplace=True)
G4_common_movies = pd.concat([G4, G4_duplicate], ignore_index=True)
G4_common_movies

/Users/elece/miniconda3/envs/baseML/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


,Emp1,Emp2,movies_in_common
0,Andy,Claude,1
1,Andy,Georgia,1
2,Andy,Pablo,1
3,Andy,Joan,1
4,Andy,Frida,1
5,Andy,Lee,1
6,Vincent,Pablo,1
7,Vincent,Frida,2
8,Claude,Georgia,3
9,Pablo,Frida,2


In [238]:
def answer_four():
        
    proj = answer_three()
    dfw = pd.DataFrame(proj.edges(data=True), columns=['emp1', 'emp2', 'movies_common'])
    dfw['movies_common'] = dfw['movies_common'].apply(lambda x: x['weight'])
    dfw_dup = dfw.copy()
    dfw_dup.rename(columns={'emp1': 'emp2', 'emp2': 'emp1'}, inplace=True)
    dfw_common_movies = pd.concat([dfw, dfw_dup], ignore_index=True, sort=False)

    df = pd.read_table('Employee_Relationships.txt', header=None,
                  names=['emp1', 'emp2', 'relationship'])

    df = df.merge(dfw_common_movies, on=['emp1', 'emp2'], how='left')
    df.fillna(0, inplace=True)

    return df.corr().iloc[0,1]

In [239]:
answer_four()

0.7883962221733476